# This is a tool for looking at HDX/MS data, LacI structure, and mutational phenotypes for particular LacI peptides in one or more functional states.

![LacI peptides and structure.](lacI_diagram_table.png)

### First choose the state(s).

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pytraj as pt
import nglview as nv
from itertools import combinations
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
import seaborn as sns
import ipywidgets as widgets
from scipy.optimize import curve_fit
from matplotlib.ticker import FormatStrFormatter


print('Choose from the following functional states: ')
CBwidget_IPTG = widgets.Checkbox(description = 'IPTG, an inducer molecule', value=True, disabled=False)
CBwidget_ONPF = widgets.Checkbox(description = 'ONPF, an anti-inducer molecule', value=True, disabled=False)
CBwidget_APO = widgets.Checkbox(description = 'APO protein', value=False, disabled=False)
CBwidget_DNA = widgets.Checkbox(description = 'operator DNA', value=True, disabled=False)
CBwidget_TMG = widgets.Checkbox(description = 'TMG, an inducer molecule', value=False, disabled=False)
CBwidget_ONPFDNA = widgets.Checkbox(description = 'ONPF and operator DNA', value=False, disabled=False)

display(CBwidget_IPTG, CBwidget_ONPF, CBwidget_APO, CBwidget_DNA, CBwidget_TMG, CBwidget_ONPFDNA)

Choose from the following functional states: 


Checkbox(value=True, description='IPTG, an inducer molecule')

Checkbox(value=True, description='ONPF, an anti-inducer molecule')

Checkbox(value=False, description='APO protein')

Checkbox(value=True, description='operator DNA')

Checkbox(value=False, description='TMG, an inducer molecule')

Checkbox(value=False, description='ONPF and operator DNA')

### Calculate HDX functions for all peptides in the chosen state(s).

In [50]:
states_list = []

if CBwidget_IPTG.value == True:
    states_list.append('IPTG')
if CBwidget_ONPF.value == True:
    states_list.append('ONPF')
if CBwidget_APO.value == True:
    states_list.append('APO')
if CBwidget_DNA.value == True:
    states_list.append('DNA')
if CBwidget_TMG.value == True:
    states_list.append('TMG')
if CBwidget_ONPFDNA.value == True:
    states_list.append('ONPFDNA')

print('Chosen states:'); print(*states_list)

res_peptides_file = pd.ExcelFile('peptide_list.xlsx')

states_dict = {}
peptide_states_list = ['peptide']
for state in states_list:
    states_dict[state] = pd.read_excel(res_peptides_file)
    peptide_states_list.append(state)
        
single_state_dict = states_dict.get(states_list[0])
all_peptides = single_state_dict[['peptide']].values.tolist()
all_peptides_2 = []
first_res = single_state_dict[['first_res']].values.tolist()
index = 0

while index < len(all_peptides):
    current_first_res = str(int(first_res[index][0]))
    all_peptides_2.append(current_first_res + ' - ' + all_peptides[index][0])
    index = index + 1

Chosen states:
IPTG ONPF DNA
['63 - LIGVA', '72 - ALHAP', '72 - ALHAPSQIVA', '72 - ALHAPSQIVAA', '72 - ALHAPSQIVAAIK', '77 - SQIVAA', '77 - SQIVAAIK', '79 - IVAAIK', '82 - AIKSRADQLGAS', '87 - ADQLGAS', '94 - VVVSM', '98 - MVERSGVE', '109 - AAVHNL', '115 - LAQRVSGL', '119 - VSGLIIN', '123 - IINYPLD', '128 - LDDQDAIA', '134 - IAVEAA', '139 - ACTNVPAL', '145 - ALFLD', '147 - FLDVSDQTPINS', '152 - DQTPIN', '158 - SIIFSH', '158 - SIIFSHEDGTRL', '164 - EDGTRLGVEH', '174 - LVAL', '174 - LVALGHQQIAL', '177 - LGHQQIALL', '182 - IALL', '185 - LAGPLS', '185 - LAGPLSSVSARL', '187 - GPLSSVSAR', '198 - LAGWHK', '204 - YLTRNQIQP', '208 - NQIQPIAEREGD', '213 - IAEREGDWSAMSG', '216 - REGDWSAMSGF', '219 - DWSAM', '220 - WSAM', '221 - SAMSGFQ', '224 - SGFQQTMQM', '228 - QTMQML', '234 - NEGIVPTAML', '243 - LVANDQMALGAM', '244 - VANDQMALGAMR', '256 - AITESGLRVGAD', '257 - ITESGLRVGAD', '266 - ADISVVGY', '270 - VVGYDD', '274 - DDTEDSSCYIPPLTTIK', '276 - TEDSSCYIPPLTTIK', '279 - SSCYIPPLTTIKQD', '293 - FRLL

## Map the peptides onto the protein structure for the chosen states and peptides.

In [58]:
############

def getIndexes(dfObj, value):
    listOfPos = []
    result = dfObj.isin([value])
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    return listOfPos

############

# create dictionary keys from peptides
# in subsequent code: add state and each TP avg exchange measurements as values for peptide keys
peptide_exchange_dict = {}
for peptide in all_peptides:
    peptide_exchange_dict[peptide[0]] = []

for state in states_dict:
    i = 0 # peptide counter
    for peptide in states_dict[state]['peptide']:
        peptide_first_res = states_dict[state]['first_res'][i]
        peptide_last_res = states_dict[state]['last_res'][i]

        peptide_file = pd.ExcelFile('~/Dropbox/Research/Data/HDX/20200816/20200819_fitting/' + str(peptide_first_res)
                                    + '_' + str(peptide_last_res) + '_' + peptide + '_fitting.xlsx')
        raw_df = pd.read_excel(peptide_file, 'Peptide SD', nrows = 15)
        
        column_vals = []
        listOfPositions = getIndexes(raw_df, state)
        for j in range(len(listOfPositions)):
            temp_string = (listOfPositions[j][1])
            position = int(temp_string[-2:])
            column_vals.append(position) # columns in spreadsheet corresponding to data label

        peptide_exchange = {}
        for element in column_vals:
            if state.lower() == (raw_df.iloc[2][element]).lower():
                peptide_exchange[state] = list(raw_df.iloc[4:13,element+2])
        
        peptide_exchange_dict[peptide].append(peptide_exchange)
        
        i = i + 1

{'LIGVA': [{'IPTG': [-0.021294718909690733, 0.14650766609882718, 0.10306643952307745, 0.07836456558781685, 0.049403747870521236, 0.11839863713809351, 0.11413969335605853, 0.13884156729131913, 0.18313458262353396]}, {'ONPF': [-0.020442930153225635, 0.028960817717295598, 0.0698466780238437, 0.10562180579218224, 0.05195911413972286, 0.10221465076670919, 0.0919931856899027, 0.09880749574104247, 0.0979557069846742]}, {'DNA': [-0.017887563884120848, 0.05110732538335459, 0.0425894378194783, 0.04599659284504819, 0.039182282793908424, 0.011073253833077916, 0.06984667802394054, 0.08006814310055337, 0.06558773424190556]}], 'ALHAP': [{'IPTG': [1.1184145373498787e-13, 0.28430890309896584, 0.2911952779144583, 0.3777668470241137, 0.6640432857846169, 1.0103295622232387, 1.48450565666503, 1.6487948844073113, 1.51598622725041]}, {'ONPF': [0.011805213969559476, 0.7525823905558097, 1.067388096409275, 1.2100344318740863, 1.168716182980796, 1.2434825381210037, 1.2405312346286979, 1.2316773241515562, 0.92670

## Plot H/D exchange functions as uptake plots for each peptide for each state.

In [ ]:
# fitting

def exchange_fit(x, a, b, c, d, e, f, g):
    max_protons = a + b + c + g - 2 - num_prolines
    d > e
    e > f
    d > f
    return max_protons - a * np.exp(-d * x) - b * np.exp(-e * x) - c * np.exp(-f * x) - g

def exchange_fit_low(x, b, c, e, f, g):
    max_protons = b + c + g - 2 - num_prolines
    e > f
    return max_protons - b * np.exp(-e * x) - c * np.exp(-f * x) - g

timepoints = [0, 30, 45, 60, 300, 1500, 3600, 7200, 14400]
trialT = np.logspace(1.5, 4.5, 10000)

num_prolines = peptide_input1[2:].count('P')
max_protons = len(peptide_input1) - 2 - num_prolines

peptide1_fits = {}
# SSR_p1_dict = {}

for key in peptide1_exchange:
    p1_index = 0
    peptide1_tps = []
    peptide1_ex = []
    for tp in peptide1_exchange.get(key):
        if not math.isnan(float(tp)):
            peptide1_tps.append(timepoints[p1_index])
            peptide1_ex.append(float(tp))
        p1_index = p1_index + 1
        
    if peptide1_ex[-1] > .5:
        
        popt, pcov = curve_fit(f = exchange_fit, xdata = peptide1_tps, ydata = peptide1_ex,
                               bounds = (0, [max_protons, max_protons, max_protons, 1, .1, .01, max_protons]),
                               maxfev = 100000)
        exchange_peptide1 = exchange_fit(trialT, *popt)
        perr = np.sqrt(np.diag(pcov))
        peptide1_fits[key] = exchange_peptide1
        
    else:

        popt, pcov = curve_fit(f = exchange_fit_low, xdata = peptide1_tps, ydata = peptide1_ex,
                               bounds = (0, [max_protons, max_protons, .1, .01, max_protons]),
                               maxfev = 100000)
        exchange_peptide1 = exchange_fit_low(trialT, *popt)
        perr = np.sqrt(np.diag(pcov))
        peptide1_fits[key] = exchange_peptide1
        
#     print(peptide_input1)
#     print(key)
#     print(popt)
    
#     residuals = peptide1_ex - exchange_fit(np.array(peptide1_tps), *popt)
#     ss_res = np.sum(residuals**2)
#     ss_tot = np.sum((peptide1_ex-np.mean(peptide1_ex))**2)
#     r_squared = 1 - (ss_res / ss_tot)

#     print("Residuals sum of squares:")
#     print(ss_res)
#     print("Total sum of squares:")
#     print(ss_tot)
#     print("R2 value")
#     print(r_squared)
#     print()


num_prolines = peptide_input2[2:].count('P')
max_protons = len(peptide_input2) - 2 - num_prolines
    
peptide2_fits = {}

for key in peptide2_exchange:
    p2_index = 0
    peptide2_tps = []
    peptide2_ex = []
    for tp in peptide2_exchange.get(key):
        if not math.isnan(float(tp)):
            peptide2_tps.append(timepoints[p2_index])
            peptide2_ex.append(float(tp))
        p2_index = p2_index + 1
        
    if peptide2_ex[-1] > .5:
        
        popt, pcov = curve_fit(f = exchange_fit, xdata = peptide2_tps, ydata = peptide2_ex,
                               bounds = (0, [max_protons, max_protons, max_protons, 1, .1, .01, max_protons]),
                               maxfev = 100000)
        exchange_peptide2 = exchange_fit(trialT, *popt)
        perr = np.sqrt(np.diag(pcov))
        peptide2_fits[key] = exchange_peptide2
        
    else:
        
        popt, pcov = curve_fit(f = exchange_fit_low, xdata = peptide2_tps, ydata = peptide2_ex,
                               bounds = (0, [max_protons, max_protons, .1, .01, max_protons]),
                               maxfev = 100000)
        exchange_peptide2 = exchange_fit_low(trialT, *popt)
        perr = np.sqrt(np.diag(pcov))
        peptide2_fits[key] = exchange_peptide2
    
#     print(peptide_input2)
#     print(key)
#     print(popt)
    
#     residuals = peptide2_ex - exchange_fit(np.array(peptide2_tps), *popt)
#     ss_res = np.sum(residuals**2)
#     ss_tot = np.sum((peptide2_ex-np.mean(peptide2_ex))**2)
#     r_squared = 1 - (ss_res / ss_tot)

#     print("Residuals sum of squares:")
#     print(ss_res)
#     print("Total sum of squares:")
#     print(ss_tot)
#     print("R2 value")
#     print(r_squared)
#     print()

# plotting
    
font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 36
       }
axes = {'titlesize' : 36,
        'titleweight' : 'bold',
        'labelsize' : 36
       }

plt.rc('font', **font)
plt.rc('axes', **axes)
plt.rc('lines', lw = 3)
color_dict = {
    'IPTG' : 'blue',
    'ONPF' : 'orange',
    'APO' : 'green',
    'DNA' : 'red',
    'TMG' : 'gray',
    'ONPFDNA' : 'purple'
    }

figure, (ax1, ax2) = plt.subplots(1, 2, figsize=(18,8))

for key in peptide1_exchange:
    ax1.plot(timepoints, peptide1_exchange.get(key), 'o', label = key, markersize = 18, alpha = 0.5, 
             color = color_dict.get(key))
    ax1.plot(trialT, peptide1_fits.get(key), '-', color = color_dict.get(key))
#     ax1.fill_between(trialT, peptide1_fits.get(key) + SSR_p1_dict.get(key), 
#                      peptide1_fits.get(key) - SSR_p1_dict.get(key), 
#                      alpha=0.2, color = color_dict.get(key))
for key in peptide2_exchange:
    ax2.plot(timepoints, peptide2_exchange.get(key), 'o', label = key, markersize = 18, alpha = 0.5,
             color = color_dict.get(key))
    ax2.plot(trialT, peptide2_fits.get(key), '-', color = color_dict.get(key))

ax1.set_ylabel('# Deuterons')
ax1.set_xlabel('Time (seconds)')
ax2.set_xlabel('Time (seconds)')
ax1.set_title(str(peptide1_first_res) + ' - ' + peptide_input1)
ax2.set_title(str(peptide2_first_res) + ' - ' + peptide_input2)
ax1.set_xscale('log')
ax2.set_xscale('log')
ax1.set_ylim(0, len(peptide_input1) - 2 - peptide_input1[2:].count('P') + 0.25)
ax2.set_ylim(0, len(peptide_input2) - 2 - peptide_input2[2:].count('P')+ 0.25)
ax1.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.legend(frameon = False, bbox_to_anchor = (1.5, 1))
# plt.show()

plt.savefig('temp_fig.pdf', bbox_inches='tight')

### Compile mutational phenotype data for Peptide 1.

In [ ]:
# This section controls the mutation information.

peptide1_range = range(peptide1_first_res, peptide1_last_res + 1, 1)

print('''
-------- Mutational phenotype data from Markiewicz et al., J. Mol Biol., 1994 --------
''')
print('Peptide 1: ' + peptide_input1)

peptide_index = 0
res_label_value = []
AA_value = []
heatmap_value = []

aa = list("YLHACDEFGIKMNPQRSTVW")

for residue in peptide1_range:
    no_effect = []
    small_effect = []
    dramatic_effect = []
    
    # print MH group
    for key, value in MH_dict.items(): 
        if residue == key:
            peptide_index = peptide_index + 1
            res_label = str(peptide_input1[peptide_index-1]) + str(residue)
            print('\n' + res_label + ': group ' + value)
            # print MH group description
            for second_key, second_value in MHdescriptions_dict.items():
                if value == second_key:
                    print(second_value)
                    
    # group and print phenotype effects of specific point mutations
    for AA in aa:
        for key, value in mutation_dict.items():
            if key[-1] == AA:
                if res_label in key:
                    if value == '+':
                        no_effect.append(key[-1])
                        heatmap_value.append(1)
                        AA_value.append(AA)
                        res_label_value.append(residue)
                    elif value == ('+-'):
                        small_effect.append(key[-1])
                        heatmap_value.append(0.25)
                        AA_value.append(AA)
                        res_label_value.append(residue)
                    elif value == ('-+'):
                        small_effect.append(key[-1])
                        heatmap_value.append(0.75)
                        AA_value.append(AA)
                        res_label_value.append(residue)
                    elif value == '-':
                        dramatic_effect.append(key[-1])
                        heatmap_value.append(0)
                        AA_value.append(AA)
                        res_label_value.append(residue)

    print('Mutations that do not affect the phenotype: ' + " ".join(x for x in no_effect))
    print('Mutations causing small effects on phenotype: ' + " ".join(x for x in small_effect))
    print('Mutations causing dramatic effects on phenotype: ' + " ".join(x for x in dramatic_effect))

heatmap_df = pd.DataFrame({'Phenotype': AA_value, 'Residue': res_label_value, 'Sensitivity': heatmap_value })
# plot it
print('\nHeatmap'
      '\nDarker colors indicate increased disruption to phenotype by mutation.'
      '\nWhite boxes - WT residue.')
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 18
plt.rcParams['font.family'] = 'DejaVu Sans'
df_wide=heatmap_df.pivot_table( index='Residue', columns='Phenotype', values='Sensitivity' )
p2=sns.heatmap( df_wide, cmap="YlGn_r" )
p2.set_title(str(peptide1_first_res) + ' - ' + peptide_input1)
plt.show()

### Compile mutational phenotype data for Peptide 2.

In [ ]:
# This section controls the mutation information.

peptide2_range = range(peptide2_first_res, peptide2_last_res + 1, 1)

print('''
-------- Mutational phenotype data from Markiewicz et al., J. Mol Biol., 1994 --------
''')
print('Peptide 2: ' + peptide_input2)

peptide_index = 0
res_label_value = []
AA_value = []
heatmap_value = []

aa = list("YLHACDEFGIKMNPQRSTVW")

for residue in peptide2_range:
    no_effect = []
    small_effect = []
    dramatic_effect = []
    
    # print MH group
    for key, value in MH_dict.items(): 
        if residue == key:
            peptide_index = peptide_index + 1
            res_label = str(peptide_input2[peptide_index-1]) + str(residue)
            print('\n' + res_label + ': group ' + value)
            # print MH group description
            for second_key, second_value in MHdescriptions_dict.items():
                if value == second_key:
                    print(second_value)
                    
    # group and print phenotype effects of specific point mutations
    for AA in aa:
        for key, value in mutation_dict.items():
            if key[-1] == AA:
                if res_label in key:
                    if value == '+':
                        no_effect.append(key[-1])
                        heatmap_value.append(1)
                        AA_value.append(AA)
                        res_label_value.append(residue)
                    elif value == ('+-'):
                        small_effect.append(key[-1])
                        heatmap_value.append(0.25)
                        AA_value.append(AA)
                        res_label_value.append(residue)
                    elif value == ('-+'):
                        small_effect.append(key[-1])
                        heatmap_value.append(0.75)
                        AA_value.append(AA)
                        res_label_value.append(residue)
                    elif value == '-':
                        dramatic_effect.append(key[-1])
                        heatmap_value.append(0)
                        AA_value.append(AA)
                        res_label_value.append(residue)

    print('Mutations that do not affect the phenotype: ' + " ".join(x for x in no_effect))
    print('Mutations causing small effects on phenotype: ' + " ".join(x for x in small_effect))
    print('Mutations causing dramatic effects on phenotype: ' + " ".join(x for x in dramatic_effect))

heatmap_df = pd.DataFrame({'Phenotype': AA_value, 'Residue': res_label_value, 'Sensitivity': heatmap_value })
# plot it
print('\nHeatmap'
      '\nDarker colors indicate increased disruption to phenotype by mutation.'
      '\nWhite boxes - WT residue.')
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 18
plt.rcParams['font.family'] = 'DejaVu Sans'
df_wide=heatmap_df.pivot_table( index='Residue', columns='Phenotype', values='Sensitivity' )
p2=sns.heatmap( df_wide, cmap="YlGn_r" )
p2.set_title(str(peptide2_first_res) + ' - ' + peptide_input2)
plt.show()